### Selected Word Group In Youtube Sentence

In [35]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [48]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File Threegram Selected 28 Word"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram_28_Word"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6
sample_num = 6

# youtube sentence condition
sent_time_start = True  # True, False
sent_time_start_time = 700 # sn

In [37]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [38]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [39]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [40]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [41]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [42]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [43]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [44]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [45]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [46]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_time_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

df_youtube_select

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
1050454,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
1050455,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
1050456,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
1050457,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [49]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,adam yine geldi
1,al bakalım dostum
2,al şunu al
3,al şunu da
4,ama asla olmaz
...,...
485,yoksa kötü mü
486,yoksa sizi de
487,zaman bugün mü
488,zaman seninle olsun


In [14]:
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [15]:
len(remove_videoid_list)

781

In [16]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
1050454,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
1050455,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
1050456,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
1050457,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [17]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [18]:
len(search_list)

412

In [19]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,sen de mi,554.389,558.032,sen de mi boncuk oldun bulut yaşasııın,69Z_fImC1v0
1,sen de mi,65.311,68.365,ya doruk sen de mi,i402pZal7Kk
2,sen de mi,650.958,652.040,sen de mi demleniyorsun,HhxZTM_WB68
3,sen de mi,407.680,413.480,herkesten beklenir ama ondan beklemezdim dedim...,Htw1hnsE63U
4,sen de mi,460.764,464.511,heeey nereye sen de mi gidiyorsun,VwcTCE17Ang
...,...,...,...,...,...
1358,hiç sorun değil,328.595,330.197,bu hiç sorun değil dave,I9CIDeQd2FQ
1359,hiç sorun değil,189.108,193.105,belki bilgisayarım yoktu hiç sorun değil ama b...,qaZnDHacYSk
1360,hiç sorun değil,195.690,204.240,yarım santim çökebilir ama hiç sorun değil yar...,rSuEJ7UHyLk
1361,hiç sorun değil,634.430,635.640,hiç sorun değil,PY5VYx2TUos


In [20]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,sen de mi,554.389000,555.347684,sen de mi boncuk oldun bulut yaşasııın,69Z_fImC1v0
1,sen de mi,66.668333,68.365000,ya doruk sen de mi,i402pZal7Kk
2,sen de mi,650.958000,651.428435,sen de mi demleniyorsun,HhxZTM_WB68
3,sen de mi,411.237333,412.088000,herkesten beklenir ama ondan beklemezdim dedim...,Htw1hnsE63U
4,sen de mi,462.126545,463.375545,heeey nereye sen de mi gidiyorsun,VwcTCE17Ang
...,...,...,...,...,...
1358,hiç sorun değil,328.734304,329.918391,bu hiç sorun değil dave,I9CIDeQd2FQ
1359,hiç sorun değil,190.236565,191.035965,belki bilgisayarım yoktu hiç sorun değil ama b...,qaZnDHacYSk
1360,hiç sorun değil,198.005625,199.519688,yarım santim çökebilir ama hiç sorun değil yar...,rSuEJ7UHyLk
1361,hiç sorun değil,634.430000,635.640000,hiç sorun değil,PY5VYx2TUos


In [21]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,sen de mi,553.789000,555.947684,sen de mi boncuk oldun bulut yaşasııın,69Z_fImC1v0
1,sen de mi,66.068333,68.965000,ya doruk sen de mi,i402pZal7Kk
2,sen de mi,650.358000,652.028435,sen de mi demleniyorsun,HhxZTM_WB68
3,sen de mi,410.637333,412.688000,herkesten beklenir ama ondan beklemezdim dedim...,Htw1hnsE63U
4,sen de mi,461.526545,463.975545,heeey nereye sen de mi gidiyorsun,VwcTCE17Ang
...,...,...,...,...,...
1358,hiç sorun değil,328.134304,330.518391,bu hiç sorun değil dave,I9CIDeQd2FQ
1359,hiç sorun değil,189.636565,191.635965,belki bilgisayarım yoktu hiç sorun değil ama b...,qaZnDHacYSk
1360,hiç sorun değil,197.405625,200.119687,yarım santim çökebilir ama hiç sorun değil yar...,rSuEJ7UHyLk
1361,hiç sorun değil,633.830000,636.240000,hiç sorun değil,PY5VYx2TUos


In [22]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,sen de mi,554,556,sen de mi boncuk oldun bulut yaşasııın,69Z_fImC1v0
1,sen de mi,66,69,ya doruk sen de mi,i402pZal7Kk
2,sen de mi,650,652,sen de mi demleniyorsun,HhxZTM_WB68
3,sen de mi,411,413,herkesten beklenir ama ondan beklemezdim dedim...,Htw1hnsE63U
4,sen de mi,462,464,heeey nereye sen de mi gidiyorsun,VwcTCE17Ang
...,...,...,...,...,...
1358,hiç sorun değil,328,331,bu hiç sorun değil dave,I9CIDeQd2FQ
1359,hiç sorun değil,190,192,belki bilgisayarım yoktu hiç sorun değil ama b...,qaZnDHacYSk
1360,hiç sorun değil,197,200,yarım santim çökebilir ama hiç sorun değil yar...,rSuEJ7UHyLk
1361,hiç sorun değil,634,636,hiç sorun değil,PY5VYx2TUos


In [23]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,sen de mi,554,556,sen de mi boncuk oldun bulut yaşasııın,69Z_fImC1v0,https://www.youtube.com/watch?v=69Z_fImC1v0&t=...
1,sen de mi,66,69,ya doruk sen de mi,i402pZal7Kk,https://www.youtube.com/watch?v=i402pZal7Kk&t=66s
2,sen de mi,650,652,sen de mi demleniyorsun,HhxZTM_WB68,https://www.youtube.com/watch?v=HhxZTM_WB68&t=...
3,sen de mi,411,413,herkesten beklenir ama ondan beklemezdim dedim...,Htw1hnsE63U,https://www.youtube.com/watch?v=Htw1hnsE63U&t=...
4,sen de mi,462,464,heeey nereye sen de mi gidiyorsun,VwcTCE17Ang,https://www.youtube.com/watch?v=VwcTCE17Ang&t=...
...,...,...,...,...,...,...
1358,hiç sorun değil,328,331,bu hiç sorun değil dave,I9CIDeQd2FQ,https://www.youtube.com/watch?v=I9CIDeQd2FQ&t=...
1359,hiç sorun değil,190,192,belki bilgisayarım yoktu hiç sorun değil ama b...,qaZnDHacYSk,https://www.youtube.com/watch?v=qaZnDHacYSk&t=...
1360,hiç sorun değil,197,200,yarım santim çökebilir ama hiç sorun değil yar...,rSuEJ7UHyLk,https://www.youtube.com/watch?v=rSuEJ7UHyLk&t=...
1361,hiç sorun değil,634,636,hiç sorun değil,PY5VYx2TUos,https://www.youtube.com/watch?v=PY5VYx2TUos&t=...


In [24]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

307

In [25]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,bir,36
1,de,27
2,bu,23
3,ne,20
4,hiç,16
...,...,...
138,belki,1
139,bence,1
140,beni,1
142,bile,1


In [26]:
# ???? 
df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
df_word_count_merge.drop_duplicates(inplace=True)
df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
df_word_count_merge.reset_index(drop=True, inplace=True)
df_word_count_merge

,word,word_count
0,bir,36
1,de,27
2,bu,23
3,ne,20
4,hiç,16
...,...,...
158,olacak,1
159,sadece,1
160,seninle,1
161,size,1


In [27]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])
set_count = set(df_result_word_manuel_count["word"])

In [28]:
set_word.difference(set_count)

{'anne',
 'asla',
 'aslında',
 'baba',
 'bizi',
 'bizim',
 'buna',
 'değilim',
 'dostum',
 'efendim',
 'eğer',
 'geliyor',
 'geri',
 'hala',
 'harika',
 'haydi',
 'ile',
 'işte',
 'küçük',
 'lazım',
 'merhaba',
 'mısın',
 'oh',
 'olabilir',
 'olmaz',
 'onları',
 'onunla',
 'pekala',
 'sanırım',
 'selam',
 'tekrar',
 'teşekkürler',
 'tüm',
 'zaten',
 'özür',
 'üzgünüm',
 'şekilde'}

In [29]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [30]:
#df_word_count_merge.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
#{sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [32]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_307_Threegram_6_Youtube_0.6s_Timeshift_Result2.xlsx']

In [33]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [34]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num